### Building GPT model


#### 1.Setup the environment

In [5]:
# # Mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# Install Java
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark
# !wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

# Unzip the file
# !tar xf spark-3.5.1-bin-hadoop3.tgz

# Setup environment for Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/home/ubuntu/spark-3.5.1-bin-hadoop3'

# Import findspark and load it
# !pip install -q findspark
import findspark
findspark.init()

# Create Spark session
from pyspark.sql import SparkSession


In [7]:
# spark

#### 2.Import libraries

In [2]:
!pip install accelerate -U
!pip install transformers[torch]

In [3]:
!pip install --upgrade openai
!pip install torch
!pip install -q pyspark==3.3.0  spark-nlp==4.3.1

In [5]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from openai import OpenAI

import json
import pickle
import urllib.request
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext

spark = sparknlp.start()

import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2LMHeadModel, AutoTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score
# from pyspark.sql.window import Window
# from pyspark import SparkContext
# from pyspark.sql import SparkSession

# from pyspark.ml import Pipeline
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
# from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
# from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
# from pyspark.ml.classification import RandomForestClassifier
# from pyspark.ml.classification import GBTClassifier

#### 3.Import data

In [6]:
# non stratified data paths
non_stratified_train = 's3a://capstone210/data/train_merged_data_conversations/'
non_stratified_test = 'workspace/capstone-210-spring2024/data/test/'

df_train = spark.read.parquet(non_stratified_train)

# train and test
df_train_nonstratified = df_train.filter((col('label') == 1) | (col('label') == 0))
df_test_nonstratified = spark.read.parquet(non_stratified_test)

24/03/06 22:01:23 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [12]:
# df_train_nonstratified.show(3)

In [13]:
# df_train_nonstratified.groupBy('label').count().show()

In [14]:
# df_test_nonstratified.groupBy('label').count().show()

Undersampling to address class imbalance

In [7]:
# function to undersample dataset automatically
def undersample(df, outcome_col, seed=1234):
  # Split dataset based on outcome
  split0_df = df.filter(col(outcome_col) == 0)
  split1_df = df.filter(col(outcome_col) == 1)
  # determine which split is major vs minor
  if (split0_df.count() > split1_df.count()):
    major_df = split0_df
    minor_df = split1_df
  else:
    minor_df = split0_df
    major_df = split1_df
  ratio = major_df.count()/minor_df.count()
  print("Ratio of major vs minor before sampling: {}".format(ratio))
  # Start under-sampling with Spark
  sampled_majority_df = major_df.sample(False, 1/ratio, seed)
  combined_df = sampled_majority_df.unionAll(minor_df)
  print(f"Final sample size: {combined_df.count()}")
  return combined_df

In [8]:
# Perform undersampling technique
df_train_UnderSampled = undersample(df_train_nonstratified, outcome_col='label')

Ratio of major vs minor before sampling: 33.05456656346749


Final sample size: 5239


In [17]:
# df_train_UnderSampled.groupBy('label').count().show()

In [18]:
# df_test_UnderSampled = undersample(df_test_nonstratified, outcome_col='label')
# df_test_UnderSampled.groupBy('label').count().show()

Split the data into train, test, val

In [9]:
# Define the split ratios (80% train, 20% validation)
train_ratio = 0.7
validation_ratio = 0.3

# Split the data into training and validation sets
df_train_split, df_val_split = df_train_UnderSampled.randomSplit([train_ratio, validation_ratio], seed=42)

# naming convention
df_test_split = df_test_nonstratified

# Cache the datasets for better performance during training
# df_train_split.cache()
# df_val_split.cache()
# df_test_split.cache()

In [20]:
# df_train_split.groupBy('label').count().show()
# df_val_split.groupBy('label').count().show()
# df_test_split.groupBy('label').count().show()

In [10]:
# Convert PySpark DataFrame to Pandas DataFrame
df_train_pd = df_train_split.select('merged_text', 'label').toPandas()
df_val_pd = df_val_split.select('merged_text', 'label').toPandas()
df_test_pd = df_test_split.select('merged_text', 'label').toPandas()

# # Tokenize the text data
# def tokenize_text(examples):
#     return tokenizer(
#         examples["merged_text"].tolist(),  # Convert DataFrame column to a list of strings
#         padding="max_length",
#         truncation=True
#     )

# # Tokenize the training, validation, and test data
# df_train_tokenized = tokenize_text(df_train_pd)
# df_val_tokenized = tokenize_text(df_val_pd)
# df_test_tokenized = tokenize_text(df_test_pd)

####4.Tokenizer and Vectorization

To incorporate text features into a machine learning model using PySpark MLlib, we need to convert the text data into numerical features before pushing it for training. Here's a general outline of the steps:

1. Tokenization: Split the text into individual words or tokens.
2. Stopwords Removal: Remove common words that do not carry much meaning (e.g., "the", "is", "and").
3. TF-IDF Vectorization: Convert the tokenized text into numerical vectors using TF-IDF (Term Frequency-Inverse Document Frequency) transformation.

In [22]:
# Load pre-trained GPT2 model and tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

#### 5.Functions for calculating performance scores

1.   Precision
2.   Recall
3.   F1



In [11]:
def calculate_precision(predictions, label_col='label', prediction_col='prediction'):
    evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='precisionByLabel',metricLabel=1)
    precision = evaluator.evaluate(predictions)
    return precision

def calculate_recall(predictions, label_col='label', prediction_col='prediction'):
    evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='recallByLabel', metricLabel=1)
    recall = evaluator.evaluate(predictions)
    return recall

def calculate_f1(predictions, label_col='label', prediction_col='prediction'):
    evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='f1', metricLabel=1)
    f1_score = evaluator.evaluate(predictions)
    return f1_score

def calculate_f1_beta(predictions, label_col='label', prediction_col='prediction'):
    evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='fMeasureByLabel', beta=3, metricLabel=1)
    f1_beta_score = evaluator.evaluate(predictions)
    return f1_beta_score

#### 6.GPT-2

In [12]:
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['merged_text']
        label = self.data.iloc[idx]['label']

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [13]:
# Define training parameters
MAX_LENGTH = 128
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
EPOCHS = 3

In [14]:
# Instantiate tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Define datasets and dataloaders
train_dataset = CustomDataset(df_train_pd, tokenizer, MAX_LENGTH)
val_dataset = CustomDataset(df_val_pd, tokenizer, MAX_LENGTH)
test_dataset = CustomDataset(df_test_pd, tokenizer, MAX_LENGTH)

In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [17]:
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*EPOCHS)

/home/ubuntu/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation loop
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, axis=1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_precision = precision_score(val_labels, val_preds, average='micro')
    val_recall = recall_score(val_labels, val_preds, average='micro')
    val_f1 = f1_score(val_labels, val_preds, average='micro')
    val_fbeta = fbeta_score(val_labels, val_preds, average='micro', beta=0.5)

    print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {avg_train_loss}, Validation Precision: {val_precision}, Validation Recall: {val_recall}, Validation F1-score: {val_f1}, Validation F1-beta: {val_fbeta}")

Epoch 1/3, Train Loss: 0.24292905157399847, Validation Precision: 0.943900848010437, Validation Recall: 0.943900848010437, Validation F1-score: 0.9439008480104369, Validation F1-beta: 0.9439008480104369
Epoch 2/3, Train Loss: 0.09348881886549422, Validation Precision: 0.9432485322896281, Validation Recall: 0.9432485322896281, Validation F1-score: 0.943248532289628, Validation F1-beta: 0.9432485322896281
Epoch 3/3, Train Loss: 0.06115720491727878, Validation Precision: 0.949119373776908, Validation Recall: 0.949119373776908, Validation F1-score: 0.949119373776908, Validation F1-beta: 0.9491193737769081


In [20]:
# testing loop
model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_precision = precision_score(test_labels, test_preds, average='binary')
test_recall = recall_score(test_labels, test_preds, average='binary')
test_f1 = f1_score(test_labels, test_preds, average='binary')
test_fbeta = fbeta_score(test_labels, test_preds, average='binary', beta=0.5)

print(f"Test Precision: {test_precision}, Test Recall: {test_recall}, Test F1-score: {test_f1}, Test F1-beta: {test_fbeta}")

Test Precision: 0.2570309718761125, Test Recall: 0.9575596816976127, Test F1-score: 0.40527645242772947, Test F1-beta: 0.3010842368640534


#### Others


In [ ]:
# # validation dataset performance
# val_precision = calculate_precision(val_predictions)
# val_recall = calculate_recall(val_predictions)
# val_f1_score = calculate_f1(val_predictions)
# val_f1_beta_score = calculate_f1_beta(val_predictions)

# print("Validation Precision:", val_precision)
# print("Validation Recall:", val_recall)
# print("Validation F1 Score:", val_f1_score)
# print("Validation F1-beta Score:", val_f1_beta_score)

In [ ]:
# # test dataset performance
# test_precision = calculate_precision(test_predictions)
# test_recall = calculate_recall(test_predictions)
# test_f1_score = calculate_f1(test_predictions)
# test_f1_beta_score = calculate_f1_beta(test_predictions)

# print("Test Precision:", test_precision)
# print("Test Recall:", test_recall)
# print("Test F1 Score:", test_f1_score)
# print("Test F1-beta Score:", test_f1_beta_score)